[**Connect 4**](https://www.kaggle.com/competitions/connect-4)

[**Connect X**](https://www.kaggle.com/competitions/connectx/leaderboard)

[debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

[round_robin](https://www.kaggle.com/code/dott1718/kdb-workshop)

[agent_ConnectX_rulebased](https://www.kaggle.com/code/dott1718/kdb-workshop) - [*dott*](https://www.kaggle.com/dott1718)

[agent_ConnectX_Cell_Swarm6](https://www.kaggle.com/code/larjeck/cell-swarm6) - [*rabbie*](https://www.kaggle.com/larjeck)

[agent_ConnectX_rb](https://www.kaggle.com/code/sanandachowdhury/connect-x) - [*Sananda Chowdhury*](https://www.kaggle.com/sanandachowdhury)

[agent_ConnectX_Swarm_Intelligence](https://www.kaggle.com/code/yegorbiryukov/connectx-swarm-intelligence) - [*Yegor Biryukov*](https://www.kaggle.com/yegorbiryukov)

In [1]:
import os
import copy
import random
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from IPython.display import clear_output

## [agent_ConnectX_rulebased_dott](https://www.kaggle.com/code/dott1718/kdb-workshop)

In [2]:
# https://www.kaggle.com/code/yegorbiryukov/connectx-rule-based
def ag_rulebased__dott__(obs, conf):
    def get_results(x, y, mark, multiplier):
        """get list of points, lowest cells and "in air" cells of a board[x][y] cell considering mark"""
        # set board[x][y] as mark
        board[x][y] = mark
        results = []
        # if some points in axis already found - axis blocked
        blocked = [False, False, False, False]
        # i is amount of marks required to add points
        for i in range(conf.inarow, 2, -1):
            # points
            p = 0
            # lowest cell
            lc = 0
            # "in air" points
            ap = 0
            # axis S -> N, only if one mark required for victory
            if i == conf.inarow and blocked[0] is False:
                (p, lc, ap, blocked[0]) = process_results(
                    p,
                    lc,
                    ap,
                    check_axis(mark, i, x, lambda z: z, y + inarow_m1, lambda z: z - 1),
                )
            # axis SW -> NE
            if blocked[1] is False:
                (p, lc, ap, blocked[1]) = process_results(
                    p,
                    lc,
                    ap,
                    check_axis(
                        mark,
                        i,
                        x - inarow_m1,
                        lambda z: z + 1,
                        y + inarow_m1,
                        lambda z: z - 1,
                    ),
                )
            # axis E -> W
            if blocked[2] is False:
                (p, lc, ap, blocked[2]) = process_results(
                    p,
                    lc,
                    ap,
                    check_axis(mark, i, x + inarow_m1, lambda z: z - 1, y, lambda z: z),
                )
            # axis SE -> NW
            if blocked[3] is False:
                (p, lc, ap, blocked[3]) = process_results(
                    p,
                    lc,
                    ap,
                    check_axis(
                        mark,
                        i,
                        x + inarow_m1,
                        lambda z: z - 1,
                        y + inarow_m1,
                        lambda z: z - 1,
                    ),
                )
            results.append((p * multiplier, lc, ap))
        # restore board[x][y] original value
        board[x][y] = 0
        return results

    def check_axis(mark, inarow, x, x_fun, y, y_fun):
        """check axis (NE -> SW etc.) for lowest cell and amounts of points and "in air" cells"""
        (x, y, axis_max_range) = get_x_y_and_axis_max_range(x, x_fun, y, y_fun)
        zeros_allowed = conf.inarow - inarow
        # lowest_cell = y
        # lowest_cell calculation turned off
        lowest_cell = 0
        for i in range(axis_max_range):
            x_temp = x
            y_temp = y
            zeros_remained = zeros_allowed
            marks = 0
            # amount of empty cells that are "in air" (don't have board bottom or mark under them)
            in_air = 0
            for j in range(conf.inarow):
                if board[x_temp][y_temp] != mark and board[x_temp][y_temp] != 0:
                    break
                elif board[x_temp][y_temp] == mark:
                    marks += 1
                # board[x_temp][y_temp] is 0
                else:
                    zeros_remained -= 1
                    if (y_temp + 1) < conf.rows and board[x_temp][y_temp + 1] == 0:
                        in_air -= 1
                #                 if y_temp > lowest_cell:
                #                     lowest_cell = y_temp
                if marks == inarow and zeros_remained == 0:
                    return (sp, lowest_cell, in_air, True)
                x_temp = x_fun(x_temp)
                y_temp = y_fun(y_temp)
                if (
                    y_temp < 0
                    or y_temp >= conf.rows
                    or x_temp < 0
                    or x_temp >= conf.columns
                ):
                    return (0, 0, 0, False)
            x = x_fun(x)
            y = y_fun(y)
        return (0, 0, 0, False)

    def get_x_y_and_axis_max_range(x, x_fun, y, y_fun):
        """set x and y inside board boundaries and get max range of axis"""
        axis_max_range = conf.inarow
        while y < 0 or y >= conf.rows or x < 0 or x >= conf.columns:
            x = x_fun(x)
            y = y_fun(y)
            axis_max_range -= 1
        return (x, y, axis_max_range)

    def process_results(p, lc, ap, axis_check_results):
        """process results of check_axis function, return lowest cell and sums of points and "in air" cells"""
        (points, lowest_cell, in_air, blocked) = axis_check_results
        if points > 0:
            if lc < lowest_cell:
                lc = lowest_cell
            ap += in_air
            p += points
        return (p, lc, ap, blocked)

    def get_best_cell(best_cell, current_cell):
        """get best cell by comparing factors of cells"""
        for i in range(len(current_cell["factors"])):
            # index 0 = points, 1 = lowest cell, 2 = "in air" cells
            for j in range(3):
                # if value of best cell factor is smaller than value of
                # the same factor in the current cell
                # best cell = current cell and break the loop,
                # don't compare lower priority factors
                if best_cell["factors"][i][j] < current_cell["factors"][i][j]:
                    return current_cell
                # if value of best cell factor is bigger than value of
                # the same factor in the current cell
                # break loop and don't compare lower priority factors
                if best_cell["factors"][i][j] > current_cell["factors"][i][j]:
                    return best_cell
        return best_cell

    def get_factors(results):
        """get list of factors represented by results and ordered by priority from highest to lowest"""
        factors = []
        for i in range(conf.inarow - 2):
            if i == 1:
                # my checker in this cell means my victory two times
                factors.append(
                    results[0][0][i] if results[0][0][i][0] > st else (0, 0, 0)
                )
                # opponent's checker in this cell means my defeat two times
                factors.append(
                    results[0][1][i] if results[0][1][i][0] > st else (0, 0, 0)
                )
                # if there are results of a cell one row above current
                if len(results) > 1:
                    # opponent's checker in cell one row above current means my defeat two times
                    factors.append(
                        results[1][1][i] if -results[1][1][i][0] > st else (0, 0, 0)
                    )
                    # my checker in cell one row above current means my victory two times
                    factors.append(
                        results[1][0][i] if -results[1][0][i][0] > st else (0, 0, 0)
                    )
                else:
                    for j in range(2):
                        factors.append((0, 0, 0))
            else:
                for j in range(2):
                    factors.append((0, 0, 0))
                for j in range(2):
                    factors.append((0, 0, 0))
            # consider only if there is no "in air" cells
            if results[0][1][i][2] == 0:
                # placing opponent's checker in this cell means opponent's victory
                factors.append(results[0][1][i])
            else:
                factors.append((0, 0, 0))
            # placing my checker in this cell means my victory
            factors.append(results[0][0][i])
            # central column priority
            factors.append((1 if i == 1 and shift == 0 else 0, 0, 0))
            # if there are results of a cell one row above current
            if len(results) > 1:
                # opponent's checker in cell one row above current means my defeat
                factors.append(results[1][1][i])
                # my checker in cell one row above current means my victory
                factors.append(results[1][0][i])
            else:
                for j in range(2):
                    factors.append((0, 0, 0))
        # if there are results of a cell two rows above current
        if len(results) > 2:
            for i in range(conf.inarow - 2):
                # my checker in cell two rows above current means my victory
                factors.append(results[2][0][i])
                # opponent's checker in cell two rows above current means my defeat
                factors.append(results[2][1][i])
        else:
            for i in range(conf.inarow - 2):
                for j in range(2):
                    factors.append((0, 0, 0))
        return factors

    # define my mark and opponent's mark
    my_mark = obs.mark
    opp_mark = 2 if my_mark == 1 else 1

    # define board as two dimensional array
    board = []
    for column in range(conf.columns):
        board.append([])
        for row in range(conf.rows):
            board[column].append(obs.board[conf.columns * row + column])

    best_cell = None
    board_center = conf.columns // 2
    inarow_m1 = conf.inarow - 1

    # standard amount of points
    sp = 1
    # "seven" pattern threshold points
    st = 1

    # start searching for best_cell from board center
    x = board_center

    # shift to right or left from board center
    shift = 0

    # searching for best_cell
    while x >= 0 and x < conf.columns:
        # find first empty cell starting from bottom of the column
        y = conf.rows - 1
        while y >= 0 and board[x][y] != 0:
            y -= 1
        # if column is not full
        if y >= 0:
            # results of current cell and cells above it
            results = []
            results.append(
                (get_results(x, y, my_mark, 1), get_results(x, y, opp_mark, 1))
            )
            # if possible, get results of a cell one row above current
            if (y - 1) >= 0:
                results.append(
                    (
                        get_results(x, y - 1, my_mark, -1),
                        get_results(x, y - 1, opp_mark, -1),
                    )
                )
            # if possible, get results of a cell two rows above current
            if (y - 2) >= 0:
                results.append(
                    (
                        get_results(x, y - 2, my_mark, 1),
                        get_results(x, y - 2, opp_mark, 1),
                    )
                )

            # list of factors represented by results
            # ordered by priority from highest to lowest
            factors = get_factors(results)

            # if best_cell is not yet found
            if best_cell is None:
                best_cell = {"column": x, "factors": factors}
            # compare values of factors in best cell and current cell
            else:
                current_cell = {"column": x, "factors": factors}
                best_cell = get_best_cell(best_cell, current_cell)

        # shift x to right or left from board center
        if shift >= 0:
            shift += 1
        shift *= -1
        x = board_center + shift

    # return index of the best cell column
    return best_cell["column"]

## [agent_ConnectX_Cell_Swarm6_rabbie](https://www.kaggle.com/code/larjeck/cell-swarm6)

In [3]:
def ag_cell_swarm_rabbie(obs, conf):
    def evaluate_cell(cell):
        """ evaluate qualities of the cell """
        cell = get_patterns(cell)
        cell = calculate_points(cell)
        for i in range(1, conf.rows):
            cell = explore_cell_above(cell, i)
        return cell
    
    def get_patterns(cell):
        """ get swarm and opponent's patterns of each axis of the cell """
        ne = get_pattern(cell["x"], lambda z : z + 1, cell["y"], lambda z : z - 1, conf.inarow)
        sw = get_pattern(cell["x"], lambda z : z - 1, cell["y"], lambda z : z + 1, conf.inarow)[::-1]
        cell["swarm_patterns"]["NE_SW"] = sw + [{"mark": swarm_mark}] + ne
        cell["opp_patterns"]["NE_SW"] = sw + [{"mark": opp_mark}] + ne
        e = get_pattern(cell["x"], lambda z : z + 1, cell["y"], lambda z : z, conf.inarow)
        w = get_pattern(cell["x"], lambda z : z - 1, cell["y"], lambda z : z, conf.inarow)[::-1]
        cell["swarm_patterns"]["E_W"] = w + [{"mark": swarm_mark}] + e
        cell["opp_patterns"]["E_W"] = w + [{"mark": opp_mark}] + e
        se = get_pattern(cell["x"], lambda z : z + 1, cell["y"], lambda z : z + 1, conf.inarow)
        nw = get_pattern(cell["x"], lambda z : z - 1, cell["y"], lambda z : z - 1, conf.inarow)[::-1]
        cell["swarm_patterns"]["SE_NW"] = nw + [{"mark": swarm_mark}] + se
        cell["opp_patterns"]["SE_NW"] = nw + [{"mark": opp_mark}] + se
        s = get_pattern(cell["x"], lambda z : z, cell["y"], lambda z : z + 1, conf.inarow)
        n = get_pattern(cell["x"], lambda z : z, cell["y"], lambda z : z - 1, conf.inarow)[::-1]
        cell["swarm_patterns"]["S_N"] = n + [{"mark": swarm_mark}] + s
        cell["opp_patterns"]["S_N"] = n + [{"mark": opp_mark}] + s
        return cell
        
    def get_pattern(x, x_fun, y, y_fun, cells_remained):
        """ get pattern of marks in direction """
        pattern = []
        x = x_fun(x)
        y = y_fun(y)
        # if cell is inside swarm's borders
        if y >= 0 and y < conf.rows and x >= 0 and x < conf.columns:
            pattern.append({
                "mark": swarm[x][y]["mark"]
            })
            # amount of cells to explore in this direction
            cells_remained -= 1
            if cells_remained > 1:
                pattern.extend(get_pattern(x, x_fun, y, y_fun, cells_remained))
        return pattern
    
    def calculate_points(cell):
        """ calculate amounts of swarm's and opponent's correct patterns and add them to cell's points """
        for i in range(conf.inarow - 2):
            # inarow = amount of marks in pattern to consider that pattern as correct
            inarow = conf.inarow - i
            swarm_points = 0
            opp_points = 0
            # calculate swarm's points and depth
            swarm_points = evaluate_pattern(swarm_points, cell["swarm_patterns"]["E_W"], swarm_mark, inarow)
            swarm_points = evaluate_pattern(swarm_points, cell["swarm_patterns"]["NE_SW"], swarm_mark, inarow)
            swarm_points = evaluate_pattern(swarm_points, cell["swarm_patterns"]["SE_NW"], swarm_mark, inarow)
            swarm_points = evaluate_pattern(swarm_points, cell["swarm_patterns"]["S_N"], swarm_mark, inarow)
            # calculate opponent's points and depth
            opp_points = evaluate_pattern(opp_points, cell["opp_patterns"]["E_W"], opp_mark, inarow)
            opp_points = evaluate_pattern(opp_points, cell["opp_patterns"]["NE_SW"], opp_mark, inarow)
            opp_points = evaluate_pattern(opp_points, cell["opp_patterns"]["SE_NW"], opp_mark, inarow)
            opp_points = evaluate_pattern(opp_points, cell["opp_patterns"]["S_N"], opp_mark, inarow)
            # if more than one mark required for victory
            if i > 0:
                # swarm_mark or opp_mark priority
                if swarm_points > opp_points:
                    cell["points"].append(swarm_points)
                    cell["points"].append(opp_points)
                else:
                    cell["points"].append(opp_points)
                    cell["points"].append(swarm_points)
            else:
                cell["points"].append(swarm_points)
                cell["points"].append(opp_points)
        return cell
                    
    def evaluate_pattern(points, pattern, mark, inarow):
        """ get amount of points, if pattern has required amounts of marks and zeros """
        # saving enough cells for required amounts of marks and zeros
        for i in range(len(pattern) - (conf.inarow - 1)):
            marks = 0
            zeros = 0
            # check part of pattern for required amounts of marks and zeros
            for j in range(conf.inarow):
                if pattern[i + j]["mark"] == mark:
                    marks += 1
                elif pattern[i + j]["mark"] == 0:
                    zeros += 1
            if marks >= inarow and (marks + zeros) == conf.inarow:
                return points + 1
        return points
    
    def explore_cell_above(cell, i):
        """ add positive or negative points from cell above (if it exists) to points of current cell """
        if (cell["y"] - i) >= 0:
            cell_above = swarm[cell["x"]][cell["y"] - i]
            cell_above = get_patterns(cell_above)
            cell_above = calculate_points(cell_above)
            # points will be positive or negative
            n = -1 if i & 1 else 1
            # if it is first cell above
            if i == 1:
                # add first 4 points of cell_above["points"] to cell["points"]
                cell["points"][2:2] = [n * cell_above["points"][1], n * cell_above["points"][0]]
                # if it is not potential "seven" pattern in cell and cell_above has more points
                if abs(cell["points"][4]) < 2 and abs(cell["points"][4]) < cell_above["points"][2]:
                    cell["points"][4:4] = [n * cell_above["points"][2]]
                    # if it is not potential "seven" pattern in cell and cell_above has more points
                    if abs(cell["points"][5]) < 2 and abs(cell["points"][5]) < cell_above["points"][3]:
                        cell["points"][5:5] = [n * cell_above["points"][3]]
                    else:
                        cell["points"][7:7] = [n * cell_above["points"][3]]
                else:
                    cell["points"][6:6] = [n * cell_above["points"][2], n * cell_above["points"][3]]
            else:
                cell["points"].extend(map(lambda z : z * n, cell_above["points"]))
        else:
            cell["points"].extend([0, 0, 0, 0])
        return cell
    
    def choose_best_cell(best_cell, current_cell):
        """ compare two cells and return the best one """
        if best_cell is not None:
            for i in range(len(best_cell["points"])):
                # compare amounts of points of two cells
                if best_cell["points"][i] < current_cell["points"][i]:
                    best_cell = current_cell
                    break
                if best_cell["points"][i] > current_cell["points"][i]:
                    break
                # if ["points"][i] of cells are equal, compare distance to swarm's center of each cell
                if best_cell["points"][i] > 0:
                    if best_cell["distance_to_center"] > current_cell["distance_to_center"]:
                        best_cell = current_cell
                        break
                    if best_cell["distance_to_center"] < current_cell["distance_to_center"]:
                        break
        else:
            best_cell = current_cell
        return best_cell

    swarm_mark = obs.mark
    opp_mark = 2 if swarm_mark == 1 else 1
    # define swarm's center
    swarm_center_horizontal = conf.columns // 2
    swarm_center_vertical = conf.rows // 2
    
    # define swarm as two dimensional array of cells
    swarm = []
    for column in range(conf.columns):
        swarm.append([])
        for row in range(conf.rows):
            cell = {
                        "x": column,
                        "y": row,
                        "mark": obs.board[conf.columns * row + column],
                        "swarm_patterns": {},
                        "opp_patterns": {},
                        "distance_to_center": abs(row - swarm_center_vertical) + abs(column - swarm_center_horizontal),
                        "points": []
                    }
            swarm[column].append(cell)
    
    best_cell = None
    # start searching for best_cell from swarm center
    x = swarm_center_horizontal
    # shift to right or left from swarm center
    shift = 0
    
    # searching for best_cell
    while x >= 0 and x < conf.columns:
        # find first empty cell starting from bottom of the column
        y = conf.rows - 1
        while y >= 0 and swarm[x][y]["mark"] != 0:
            y -= 1
        # if column is not full
        if y >= 0:
            # current cell evaluates its own qualities
            current_cell = evaluate_cell(swarm[x][y])
            # current cell compares itself against best cell
            best_cell = choose_best_cell(best_cell, current_cell)
                        
        # shift x to right or left from swarm center
        if shift >= 0:
            shift += 1
        shift *= -1
        x = swarm_center_horizontal + shift

    # return index of the best cell column
    return best_cell["x"]

## [agent_ConnectX_rb_SanandaChowdhury](https://www.kaggle.com/code/sanandachowdhury/connect-x)

In [4]:
def ag_Sananda_Chowdhury(observation, configuration):
    
    from random import choice
    
    # me:me_or_enemy=1, enemy:me_or_enemy=2
    def check_vertical_chance(me_or_enemy):
        for i in range(0, 7):
            if observation.board[i+7*5] == me_or_enemy \
            and observation.board[i+7*4] == me_or_enemy \
            and observation.board[i+7*3] == me_or_enemy \
            and observation.board[i+7*2] == 0:
                return i
            elif observation.board[i+7*4] == me_or_enemy \
            and observation.board[i+7*3] == me_or_enemy \
            and observation.board[i+7*2] == me_or_enemy \
            and observation.board[i+7*1] == 0:
                return i
            elif observation.board[i+7*3] == me_or_enemy \
            and observation.board[i+7*2] == me_or_enemy \
            and observation.board[i+7*1] == me_or_enemy \
            and observation.board[i+7*0] == 0:
                return i
        # no chance
        return -99
    
    # me:me_or_enemy=1, enemy:me_or_enemy=2
    def check_horizontal_chance(me_or_enemy):
        chance_cell_num = -99
        for i in [0,7,14,21,28,35]:
            for j in range(0, 4):
                val_1 = i+j+0
                val_2 = i+j+1
                val_3 = i+j+2
                val_4 = i+j+3
                if sum([observation.board[val_1] == me_or_enemy, \
                        observation.board[val_2] == me_or_enemy, \
                        observation.board[val_3] == me_or_enemy, \
                        observation.board[val_4] == me_or_enemy]) == 3:
                    for k in [val_1,val_2,val_3,val_4]:
                        if observation.board[k] == 0:
                            chance_cell_num = k
                            # bottom line
                            for l in range(35, 42):
                                if chance_cell_num == l:
                                    return l - 35
                            # others
                            if observation.board[chance_cell_num+7] != 0:
                                return chance_cell_num % 7
        # no chance
        return -99
    
    # me:me_or_enemy=1, enemy:me_or_enemy=2
    def check_slanting_chance(me_or_enemy, lag, cell_list):
        chance_cell_num = -99
        for i in cell_list:
            val_1 = i+lag*0
            val_2 = i+lag*1
            val_3 = i+lag*2
            val_4 = i+lag*3
            if sum([observation.board[val_1] == me_or_enemy, \
                    observation.board[val_2] == me_or_enemy, \
                    observation.board[val_3] == me_or_enemy, \
                    observation.board[val_4] == me_or_enemy]) == 3:
                for j in [val_1,val_2,val_3,val_4]:
                    if observation.board[j] == 0:
                        chance_cell_num = j
                        # bottom line
                        for k in range(35, 42):
                            if chance_cell_num == k:
                                return k - 35
                        # others
                        if chance_cell_num != -99 \
                        and observation.board[chance_cell_num+7] != 0:
                            return chance_cell_num % 7
        # no chance
        return -99
    
    def check_horizontal_first_enemy_chance():
        # enemy's chance
        if observation.board[38] == enemy_num:
            if sum([observation.board[39] == enemy_num, observation.board[40] == enemy_num]) == 1 \
            and observation.board[37] == 0:
                for i in range(39, 41):
                    if observation.board[i] == 0:
                        return i - 35
            if sum([observation.board[36] == enemy_num, observation.board[37] == enemy_num]) == 1 \
            and observation.board[39] == 0:
                for i in range(36, 38):
                    if observation.board[i] == 0:
                        return i - 35
        # no chance
        return -99

    def check_first_or_second():
        count = 0
        for i in observation.board:
            if i != 0:
                count += 1
        # first
        if count % 2 != 1:
            my_num = 1
            enemy_num = 2
        # second
        else:
            my_num = 2
            enemy_num = 1
        return my_num, enemy_num
    
    # check first or second
    my_num, enemy_num = check_first_or_second()
    
    def check_my_chances():
        # check my virtical chance
        result = check_vertical_chance(my_num)
        if result != -99:
            return result
        # check my horizontal chance
        result = check_horizontal_chance(my_num)
        if result != -99:
            return result
        # check my slanting chance 1 (up-right to down-left)
        result = check_slanting_chance(my_num, 6, [3,4,5,6,10,11,12,13,17,18,19,20])
        if result != -99:
            return result
        # check my slanting chance 2 (up-left to down-right)
        result = check_slanting_chance(my_num, 8, [0,1,2,3,7,8,9,10,14,15,16,17])
        if result != -99:
            return result
        # no chance
        return -99
    
    def check_enemy_chances():
        # check horizontal first chance
        result = check_horizontal_first_enemy_chance()
        if result != -99:
            return result
        # check enemy's vertical chance
        result = check_vertical_chance(enemy_num)
        if result != -99:
            return result
        # check enemy's horizontal chance
        result = check_horizontal_chance(enemy_num)
        if result != -99:
            return result
        # check enemy's slanting chance 1 (up-right to down-left)
        result = check_slanting_chance(enemy_num, 6, [3,4,5,6,10,11,12,13,17,18,19,20])
        if result != -99:
            return result
        # check enemy's slanting chance 2 (up-left to down-right)
        result = check_slanting_chance(enemy_num, 8, [0,1,2,3,7,8,9,10,14,15,16,17])
        if result != -99:
            return result
        # no chance
        return -99
    
    if my_num == 1:
        result = check_my_chances()
        if result != -99:
            return result
        result = check_enemy_chances()
        if result != -99:
            return result
    if my_num == 2:
        result = check_enemy_chances()
        if result != -99:
            return result
        result = check_my_chances()
        if result != -99:
            return result
    
    # select center as priority (3 > 2 > 4 > 1 > 5 > 0 > 6)
    # column 3
    if observation.board[24] != enemy_num \
    and observation.board[17] != enemy_num \
    and observation.board[10] != enemy_num \
    and observation.board[3] == 0:
        return 3
    # column 2
    elif observation.board[23] != enemy_num \
    and observation.board[16] != enemy_num \
    and observation.board[9] != enemy_num \
    and observation.board[2] == 0:
        return 2
    # column 4
    elif observation.board[25] != enemy_num \
    and observation.board[18] != enemy_num \
    and observation.board[11] != enemy_num \
    and observation.board[4] == 0:
        return 4
    # column 1
    elif observation.board[22] != enemy_num \
    and observation.board[15] != enemy_num \
    and observation.board[8] != enemy_num \
    and observation.board[1] == 0:
        return 1
    # column 5
    elif observation.board[26] != enemy_num \
    and observation.board[19] != enemy_num \
    and observation.board[12] != enemy_num \
    and observation.board[5] == 0:
        return 5
    # column 0
    elif observation.board[21] != enemy_num \
    and observation.board[14] != enemy_num \
    and observation.board[7] != enemy_num \
    and observation.board[0] == 0:
        return 0
    # column 6
    elif observation.board[27] != enemy_num \
    and observation.board[20] != enemy_num \
    and observation.board[13] != enemy_num \
    and observation.board[6] == 0:
        return 6
    # random
    else:
        return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

## [agent_ConnectX_Swarm_Intelligence_YegorBiryukov](https://www.kaggle.com/code/yegorbiryukov/connectx-swarm-intelligence)

In [5]:
def ag_SwarmIntelligence(obs, conf):
    def send_scout_carrier(x, y):
        """ send scout carrier to explore current cell and, if possible, cell above """
        points = send_scouts(x, y)
        # if cell above exists
        if y > 0:
            cell_above_points = send_scouts(x, y - 1)
            # cell above points have lower priority
            if points < m1 and points < (cell_above_points - 1):
                # current cell's points will be negative
                points -= cell_above_points
        return points
    
    def send_scouts(x, y):
        """ send scouts to get points from all axes of the cell """
        axes = explore_axes(x, y)
        points = combine_points(axes)
        return points
        
    def explore_axes(x, y):
        """
            find points, marks, zeros and amount of in_air cells of all axes of the cell,
            "NE" = North-East etc.
        """
        return {
            "NE -> SW": [
                explore_direction(x, lambda z : z + 1, y, lambda z : z - 1),
                explore_direction(x, lambda z : z - 1, y, lambda z : z + 1)
            ],
            "E -> W": [
                explore_direction(x, lambda z : z + 1, y, lambda z : z),
                explore_direction(x, lambda z : z - 1, y, lambda z : z)
            ],
            "SE -> NW": [
                explore_direction(x, lambda z : z + 1, y, lambda z : z + 1),
                explore_direction(x, lambda z : z - 1, y, lambda z : z - 1)
            ],
            "S -> N": [
                explore_direction(x, lambda z : z, y, lambda z : z + 1),
                explore_direction(x, lambda z : z, y, lambda z : z - 1)
            ]
        }
    
    def explore_direction(x, x_fun, y, y_fun):
        """ get points, mark, zeros and amount of in_air cells of this direction """
        # consider only opponents mark
        mark = 0
        points = 0
        zeros = 0
        in_air = 0
        for i in range(one_mark_to_win):
            x = x_fun(x)
            y = y_fun(y)
            # if board[x][y] is inside board's borders
            if y >= 0 and y < conf.rows and x >= 0 and x < conf.columns:
                # mark of the direction will be the mark of the first non-empty cell
                if mark == 0 and board[x][y] != 0:
                    mark = board[x][y]
                # if board[x][y] is empty
                if board[x][y] == 0:
                    zeros += 1
                    if (y + 1) < conf.rows and board[x][y + 1] == 0:
                        in_air += 1
                elif board[x][y] == mark:
                    points += 1
                # stop searching for marks in this direction
                else:
                    break
        return {
            "mark": mark,
            "points": points,
            "zeros": zeros,
            "in_air": in_air
        }
    
    def combine_points(axes):
        """ combine points of different axes """
        points = 0
        # loop through all axes
        for axis in axes:
            # if mark in both directions of the axis is the same
            # or mark is zero in one or both directions of the axis
            if (axes[axis][0]["mark"] == axes[axis][1]["mark"]
                    or axes[axis][0]["mark"] == 0 or axes[axis][1]["mark"] == 0):
                # combine points of the same axis
                points += evaluate_amount_of_points(
                              axes[axis][0]["points"] + axes[axis][1]["points"],
                              axes[axis][0]["zeros"] + axes[axis][1]["zeros"],
                              axes[axis][0]["in_air"] + axes[axis][1]["in_air"],
                              m1,
                              m2,
                              axes[axis][0]["mark"]
                          )
            else:
                # if marks in directions of the axis are different and none of those marks is 0
                for direction in axes[axis]:
                    points += evaluate_amount_of_points(
                                  direction["points"],
                                  direction["zeros"],
                                  direction["in_air"],
                                  m1,
                                  m2,
                                  direction["mark"]
                              )
        return points
    
    def evaluate_amount_of_points(points, zeros, in_air, m1, m2, mark):
        """ evaluate amount of points in one direction or entire axis """
        # if points + zeros in one direction or entire axis >= one_mark_to_win
        # multiply amount of points by one of the multipliers or keep amount of points as it is
        if (points + zeros) >= one_mark_to_win:
            if points >= one_mark_to_win:
                points *= m1
            elif points == two_marks_to_win:
                points = points * m2 + zeros - in_air
            else:
                points = points + zeros - in_air
        else:
            points = 0
        return points


    #################################################################################
    # one_mark_to_win points multiplier
    m1 = 100
    # two_marks_to_win points multiplier
    m2 = 10
    # define swarm's mark
    swarm_mark = obs.mark
    # define opponent's mark
    opp_mark = 2 if swarm_mark == 1 else 1
    # define one mark to victory
    one_mark_to_win = conf.inarow - 1
    # define two marks to victory
    two_marks_to_win = conf.inarow - 2
    # define board as two dimensional array
    board = []
    for column in range(conf.columns):
        board.append([])
        for row in range(conf.rows):
            board[column].append(obs.board[conf.columns * row + column])
    # define board center
    board_center = conf.columns // 2
    # start searching for the_column from board center
    x = board_center
    # shift to left/right from board center
    shift = 0
    # THE COLUMN !!!
    the_column = {
        "x": x,
        "points": float("-inf")
    }
    
    # searching for the_column
    while x >= 0 and x < conf.columns:
        # find first empty cell starting from bottom of the column
        y = conf.rows - 1
        while y >= 0 and board[x][y] != 0:
            y -= 1
        # if column is not full
        if y >= 0:
            # send scout carrier to get points
            points = send_scout_carrier(x, y)
            # evaluate which column is THE COLUMN !!!
            if points > the_column["points"]:
                the_column["x"] = x
                the_column["points"] = points
        # shift x to right or left from swarm center
        shift *= -1
        if shift >= 0:
            shift += 1
        x = board_center + shift
    
    # Swarm's final decision :)
    return the_column["x"]

In [6]:
from kaggle_environments import make, evaluate

env__6_7 = make("connectx",configuration={"inarow":4,"columns":7,"rows":6})
env__8_8 = make("connectx",configuration={"inarow":4,"columns":8,"rows":8})

No pygame installed, ignoring import


In [7]:
env__6_7.run([ag_cell_swarm_rabbie, ag_SwarmIntelligence])
env__6_7.render(mode="ipython", width=500, height=450)

In [8]:
env__6_7.run([ag_cell_swarm_rabbie, ag_rulebased__dott__])
env__6_7.render(mode="ipython", width=500, height=450)

In [9]:
env__6_7.run([ag_rulebased__dott__, ag_SwarmIntelligence])
env__6_7.render(mode="ipython", width=500, height=450)

In [10]:
env__6_7.run([ag_Sananda_Chowdhury, ag_Sananda_Chowdhury])
env__6_7.render(mode="ipython", width=500, height=450)

In [11]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(ag_rulebased__dott__,"submission_ag_rulebased__dott__.py")
write_agent_to_file(ag_cell_swarm_rabbie,"submission_ag_cell_swarm_rabbie.py")
write_agent_to_file(ag_Sananda_Chowdhury,"submission_ag_Sananda_Chowdhury.py")
write_agent_to_file(ag_SwarmIntelligence,"submission_ag_SwarmIntelligence.py")

<function ag_rulebased__dott__ at 0x7f41d2cfc1f0> written to submission_ag_rulebased__dott__.py
<function ag_cell_swarm_rabbie at 0x7f41d2cfc280> written to submission_ag_cell_swarm_rabbie.py
<function ag_Sananda_Chowdhury at 0x7f41d2cfd6c0> written to submission_ag_Sananda_Chowdhury.py
<function ag_SwarmIntelligence at 0x7f41d2cfd5a0> written to submission_ag_SwarmIntelligence.py


#### [round_robin](https://www.kaggle.com/code/dott1718/kdb-workshop)

In [12]:
import copy
import random
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

def create(): 
    return make("connectx", configuration={"inarow":4,"columns":7,"rows":6})
       
def run_game(agent1, agent2, shuffle=False):
    env = create()
    swap = shuffle and random.choice([True, False])
    if swap:
        env.run([agent2, agent1])
    else:
        env.run([agent1, agent2])

    if env.steps[-1][0].status == "TIMEOUT":
        rew1 = 0
        rew2 = 1
    elif env.steps[-1][1].status == "TIMEOUT":
        rew2 = 0
        rew1 = 1
    else:
        rew1 = (env.steps[-1][0].reward + 1.0) / 2
        rew2 = (env.steps[-1][1].reward + 1.0) / 2

    time1 = max(0, env.steps[-1][0].observation.remainingOverageTime)
    time2 = max(0, env.steps[-1][1].observation.remainingOverageTime)

    if swap:
        return rew2, rew1, time2, time1, len(env.steps)
    else:
        return rew1, rew2, time1, time2, len(env.steps)


def run_games(agent1, agent2, n_games=1, n_jobs=-1, shuffle=True):
    if n_jobs != 1:
        game_res = Parallel(
            n_jobs=-1, temp_folder="/tmp", max_nbytes=None, backend="loky"
        )(delayed(run_game)(agent1, agent2, shuffle) for _ in range(n_games))
    else:
        game_res = [run_game(agent1, agent2, shuffle) for _ in range(n_games)]

    return [np.mean([x[i] for x in game_res]) for i in range(5)]


def round_robin(agents, n_games=1, n_jobs=-1):
    res = {
        "name": [],
        "win_rate": [],
        "rem_time": [],
    }
    for i in range(len(agents)):
        for j in range(len(agents)):
            if j <= i:
                continue
            game_res = run_games(agents[i], agents[j], n_games, n_jobs)
            res["name"].append(agents[i] if type(agents[i]) == str else agents[i].__name__)
            res["win_rate"].append(game_res[0])
            res["rem_time"].append(game_res[2])
            res["name"].append(agents[j] if type(agents[j]) == str else agents[j].__name__)
            res["win_rate"].append(game_res[1])
            res["rem_time"].append(game_res[3])
    return (
        pd.DataFrame(res)
        .groupby("name")
        .mean()
        .reset_index()
        .sort_values(["win_rate", "rem_time"], ascending=False)
        .reset_index(drop=True)
    )

#### [debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

In [13]:
def debugger_c4(list_of_agents_to_trace):
    map4q = '''
         01,02,03,04; 31,32,33,34;                 32,33,34,35; 33,34,35,36; 
           40,41,42,43; 41,42,43,44;             42,43,44,45; 43,44,45,46; 
             50,51,52,53; 51,52,53,54;         52,53,54,55; 30,31,32,33;
               04,14,24,34; 14,24,34,44;     24,34,44,54; 05,15,25,35; 
                 06,16,26,36; 03,12,21,30; 15,25,35,45; 25,35,45,55; 
                   16,26,36,46; 26,36,46,56;          23,34,45,56;
                     04,13,22,31; 13,22,31,40;      03,14,25,36; 
                       05,14,23,32; 14,23,32,41;  20,30,40,50; 
                         06,15,24,33; 15,24,33,42;
                           24,33,42,51; 23,32,41,50;
                             03,13,23,33; 13,23,33,43; 
                               16,25,34,43; 25,34,43,52; 
                                 11,22,33,44; 11,21,31,41;
                     01,12,23,34;  02,13,24,35; 12,23,34,45; 
                   00,11,22,33;      26,35,44,53; 23,33,43,53;              
                 02,12,22,32;          21,32,43,54; 22,33,44,55; 
               00,10,20,30; 22,23,24,25; 21,31,41,51; 13,24,35,46;
             01,11,21,31; 10,20,30,40;     12,22,32,42; 22,32,42,52;
           00,01,02,03; 10,21,32,43;         02,03,04,05; 03,04,05,06;
         10,11,12,13; 11,12,13,14;             12,13,14,15; 13,14,15,16;
       20,21,22,23; 21,22,23,24;                 23,24,25,26; 20,31,42,53; 53,54,55,56'''
    
    # -----------------------------------------------------------------------------------------------
    start_games_at_away_first_agent = False # True # => The first agent from the list will start his game either “from home” or “away”,
    only_observation_first_agent    = False # True # => Everyone will play two games with the first: one “at home”, the other “away”, or everyone will play with everyone
    # -----------------------------------------------------------------------------------------------
    class Obs:
        def __init__(self, board, mark):
            self.board = board
            self.mark = mark
    # -----------------------------------------------------------------------------------------------
    class Config:
        def __init__(self, rows, columns, inarow):
            self.rows = rows
            self.columns = columns
            self.inarow = inarow
    # -----------------------------------------------------------------------------------------------
    import time
    import numpy as np
    # -----------------------------------------------------------------------------------------------
    Rows, Cols = 6, 7
    config = Config(Rows, Cols, 4)
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    # -----------------------------------------------------------------------------------------------
    def yx(s): return AT2[int(s[0])][int(s[1])]
    Ss = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMap4q = [list(map(yx, s)) for s in Ss]
    # -----------------------------------------------------------------------------------------------
    def gyx(s): return [int(s[0]), int(s[1])]
    gSs = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    gMap4q = [list(map(gyx, s)) for s in gSs]
    # -----------------------------------------------------------------------------------------------
    def inject(col, mark, board):
        i_krai = config.columns * (config.rows - 1) + col
        for i in range(i_krai, -1, -7):
            if board[i] == 0:
                new_board = board.copy()
                new_board[i] = mark
                return new_board
        return board
    # -----------------------------------------------------------------------------------------------
    def win_4o_in(board):
        grid = np.asarray(board).reshape(6,7)
        for i in gMap4q:
            v = grid[i[0][0],i[0][1]] * grid[i[1][0],i[1][1]] * grid[i[2][0],i[2][1]] * grid[i[3][0],i[3][1]]
            if v == 1 or v == 16: return True
        return False
    # -----------------------------------------------------------------------------------------------
    def tsum(reagent):
        home = sum(reagent["games"][0]) if len(reagent["games"][0])>0 else 0
        away = sum(reagent["games"][1]) if len(reagent["games"][1])>0 else 0
        return home + away
    # -----------------------------------------------------------------------------------------------
    def ttime(reagent, gt):
        time = reagent["time"][0]  if len(reagent["time"])>0 else 0
        return [round(time+gt, 2)]
    # -----------------------------------------------------------------------------------------------
    def tmoves(reagent, ms):
        moves = reagent["moves"][0] if len(reagent["moves"])>0 else 0
        return [moves+ms]
    # -----------------------------------------------------------------------------------------------
    def tper1move(reagent):
        time = reagent["time"][0]   if len(reagent["time"]) > 0 else 0
        moves = reagent["moves"][0] if len(reagent["moves"])> 0 else 0
        return [round(time / moves, 1)]
    # -----------------------------------------------------------------------------------------------
    def print_to_clip(board, ims, i, attack, defense, stime1, stime2, s1, s2):
        print("1.{0} [{1}]   2.{2} [{3}]".format(name(attack), s1, name(defense), s2),
              "  1.Time =", round(stime1, 2), '  2.Time =', round(stime2, 2), '  q.moves:', i,
              "  1.speed =", round(stime1 / i, 1), "  2.speed =", round(stime2 / i, 1),
              "  \n\nGame:", str(ims).replace(", ", ","),
              "  \n\nBoard:", str(board).replace(", ", ","),"\n")
        bn2d = np.array(board).reshape(Rows, Cols)
        for r in range(len(bn2d)): print(bn2d[r])
        print('{0}{1}'.format('\n',"="*121))
    # -----------------------------------------------------------------------------------------------
    def rec(total_points, _A, _D, ia, id, stime1, stime2):
        reatta = total_points[name(Attack)]
        redefe = total_points[name(Defense)]
        reatta["games"][0].extend([_A])
        redefe["games"][1].extend([_D])
        reatta["T"] = [tsum(reatta)]
        redefe["T"] = [tsum(redefe)]
        reatta["time"] = ttime(reatta, stime1)
        redefe["time"] = ttime(redefe, stime2)
        reatta["moves"] = tmoves(reatta, ia)
        redefe["moves"] = tmoves(redefe, id)
        reatta["speed"] = tper1move(reatta)
        redefe["speed"] = tper1move(redefe)
    # -----------------------------------------------------------------------------------------------
    def agent_work(Agent, board, mark, config):
        obs = Obs(board, mark)
        start = time.time()
        im = Agent(obs, config)
        t = time.time() - start
        board_next = inject(im, obs.mark, board)
        return im, t, board_next
    # -----------------------------------------------------------------------------------------------
    def write_Connect4(ims):
        #     with open("Connect4.txt", "w") as file:
        #         file.write(str(ims))
        pass 
    # -----------------------------------------------------------------------------------------------
    def write_uraConnect(ura_ims, Attack, Defense):
        #     try:
        #         with open("uraConnect.txt", "w") as file:
        #             stf = str(ura_ims) + '\n' + name(Attack) + ' - ' + name(Defense)
        #             file.write(stf)
        #     finally: return
        pass
    # -----------------------------------------------------------------------------------------------
    def name(agent): return agent.__name__.replace("my_","")
    # -----------------------------------------------------------------------------------------------

    Agents = list_of_agents_to_trace # [ag1,ag2,..,agn]

    observatory = Agents[0] if only_observation_first_agent and len(Agents) > 1 else None

    total_points = { name(agent):{"T":[],"games":[[],[]],"time":[],"moves":[],"speed":[]} for agent in Agents }

    ig, board__0 = 1, [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for home in Agents:
        for away in Agents:
            if home!=away and (observatory==None or observatory!=None and (observatory==home or observatory==away)):
                if ig==1:
                    print("\n")
                    print(ig-1, ":")
                    print('----------------')
                    for itp in total_points.items(): print(itp)
                    ig += 1
                stime1, stime2, imts1, imts2, ims, imt, = 0, 0, [], [], [], [[], []]
                board__1 = board__0
                # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                if start_games_at_away_first_agent:
                    Attack, Defense = away, home
                else:
                    Attack, Defense = home, away
                # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                total_points_Attack, total_points_Defense = [], []
                print("--------------------------")
                print(Attack.__name__, "  vs  ", Defense.__name__)
                print("-----------------------------------")
                for i in range(1,22):
                    im1, t1, board__2  = agent_work(Attack, board__1, 1, config)
                    stime1 +=t1
                    imt[0] = str(round(t1,0))
                    ims.append(im1)
                    imts1.append(imt[0])
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__2):
                        rec(total_points, 3, 0, i, i-1, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   1 1  1   1    1     1     ', Attack.__name__, "     1     1    1   1  1 1")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip (board__2, ims, i, Attack, Defense, stime1, stime2, 3,0)
                        write_Connect4 (ims)
                        write_uraConnect (ims, Attack, Defense)
                        time.sleep(1)
                        break
                    im2, t2, board__1 = agent_work(Defense, board__2, 2, config)
                    stime2 +=t2
                    imt[1] = str(round(t2, 0))
                    ims.append(im2)
                    imts2.append(imt[1])
                    print('({0}) . {1} > {2}  {3} < {4} . ({5}) --- {6} : {7} '
                          .format(round(t1,1), name(Attack), im1, im2, name(Defense), round(t2,1), i, str(ims).replace(", ",",")))
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__1):
                        rec(total_points, 0, 3, i, i, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   2 2  2   2    2     2     ', Defense.__name__,"     2     2    2   2  2 2")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 0,3)
                        write_Connect4 (ims)
                        write_uraConnect(ims, Attack, Defense)
                        time.sleep(1)
                        break
                if not win_4o_in(board__1) and not win_4o_in(board__2):
                    rec(total_points, 1, 2, i, i, stime1, stime2)
                    print('\nDRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W')
                    print('  0 0  0   0    0     ', Attack.__name__, " - ", Defense.__name__, "     0    0   0  0 0")
                    print('DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W\n')
                    print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 1,2)
                    write_Connect4 (ims)
                    write_uraConnect(ims, Attack, Defense)
                    time.sleep(3)
                print("\n")
                print(ig,":\n") # , ": ", total_points,
                for itp in total_points.items():
                    print(itp)
                print("\n")
                ig +=1

In [14]:
debugger_c4([
    ag_rulebased__dott__, 
    ag_cell_swarm_rabbie, 
    ag_Sananda_Chowdhury, 
    ag_SwarmIntelligence
])



0 :
----------------
('ag_rulebased__dott__', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('ag_cell_swarm_rabbie', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('ag_Sananda_Chowdhury', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('ag_SwarmIntelligence', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
--------------------------
ag_rulebased__dott__   vs   ag_cell_swarm_rabbie
-----------------------------------
(0.0) . ag_rulebased__dott__ > 3  3 < ag_cell_swarm_rabbie . (0.0) --- 1 : [3,3] 
(0.0) . ag_rulebased__dott__ > 3  3 < ag_cell_swarm_rabbie . (0.0) --- 2 : [3,3,3,3] 
(0.0) . ag_rulebased__dott__ > 3  3 < ag_cell_swarm_rabbie . (0.0) --- 3 : [3,3,3,3,3,3] 
(0.0) . ag_rulebased__dott__ > 2  4 < ag_cell_swarm_rabbie . (0.0) --- 4 : [3,3,3,3,3,3,2,4] 
(0.0) . ag_rulebased__dott__ > 1  0 < ag_cell_swarm_rabbie . (0.0) --- 5 : [3,3,3,3,3,3,2,4,1,0] 
(0.0) . ag_rulebased__dott__ > 4  1 < ag_ce

## bot battle

In [15]:
round_robin(agents=[
    "random",
    'negamax',
    ag_rulebased__dott__, 
    ag_cell_swarm_rabbie, 
    ag_Sananda_Chowdhury, 
    ag_SwarmIntelligence
], n_games=10)

,name,win_rate,rem_time
0,ag_cell_swarm_rabbie,0.90,60.0
1,ag_rulebased__dott__,0.80,60.0
2,ag_SwarmIntelligence,0.58,60.0
3,ag_Sananda_Chowdhury,0.48,60.0
4,negamax,0.24,60.0
5,random,0.00,60.0
